In [4]:
#from pyscf import gto, scf, md
import re
import pickle
import numpy as np
import os 
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt 
from datetime import datetime
from mpl_toolkits.mplot3d import Axes3D

In [2]:
CONSTANTS = {
    "width_ticks": 10,
    "length_ticks": 20
}

In [3]:
def run_md_simulation():
    h2 = gto.Mole()
    h2.atom = [['H', (0.7, 0, 0)], ['H', (-0.7, 0, 0)]]
    h2.basis = 'ccpvdz'
    h2.unit = 'B'
    h2.build() 
    h2._atom
    mf = scf.RHF(h2)
    mf.kernel()
    mycas = mf.CASSCF(2, 2)
    myscanner = mycas.nuc_grad_method().as_scanner()

    # Generate the integrator
    # sets the time step to 5 a.u. and will run for 100 steps
    # or for 50 a.u.
    myintegrator = md.NVE(myscanner,
                                dt=1,
                                steps=1200,
                                energy_output="BOMD.md.energies",
                                trajectory_output="BOMD.md.xyz",
                                verbose=0).run()

    # Note that we can also just pass the CASSCF object directly to
    # generate the integrator and it will automatically convert it to a scanner
    # myintegrator = pyscf.md.NVE(mycas, dt=5, steps=100)

    # Close the file streams for the energy and trajectory.
    myintegrator.energy_output.close()
    myintegrator.trajectory_output.close()

In [4]:
def grid_coordinates(ticks=30):
    raw_data = open("./BOMD.md.xyz", "r").read() + "2\n"
    raw_data = raw_data.split("MD Time")[1:]
    R = []
    for n in range(len(raw_data)):
        data_point = raw_data[n]
        data_point = data_point.split("\n")[1:-2]
        data_point = [re.sub(r'\s+', ',', coords).split(",") for coords in data_point]
        data_point = [[x, y, z] for [atom, x, y, z] in data_point]
        R.append(data_point)
    R = np.array(R, dtype=np.float32)
    R_reshaped = R.reshape((len(raw_data) * 2, 3))
    max_abs_value = 2 * np.sqrt((R_reshaped * R_reshaped).sum(axis=-1)).max()
    # since the hydrogen molecule is always centered around the origin, grid axis can be chosen to be symmetrical wrt origin
    array = np.linspace(-max_abs_value / 2, max_abs_value, ticks)
    coords = np.array([[x, y, 0] for x in array for y in array])
    data_dict = {"R": R, "coords": coords}
    pickle.dump(data_dict, open(f"./md_h2_R_coords.npz", "wb"))

In [5]:
def regular_sphere_points(radius, center, azimut_angle):
    points = []
    n_azimut = np.pi / azimut_angle
    d_azi = radius * np.pi / n_azimut
    for theta_azi in np.arange(azimut_angle, np.floor(np.pi / azimut_angle) * azimut_angle, azimut_angle):
        r_polar = np.sin(theta_azi) * radius
        polar_angle_increment = d_azi / r_polar
        for phi_pol in np.arange(0, np.floor(2 * np.pi / polar_angle_increment) * polar_angle_increment, polar_angle_increment):
            x = radius * np.sin(theta_azi) * np.cos(phi_pol) + center[0]
            y = radius * np.sin(theta_azi) * np.sin(phi_pol) + center[1]
            z = radius * np.cos(theta_azi) + center[2]
            points.append([x, y, z]) 
    points = np.array(points).T.tolist()
    [x, y, z] = points
    return points

In [6]:
def rectangle_coordinates(R1, R2, width_ticks, length_ticks):
    # it is assumed that the center lies within the z=0-plane
    R1 = np.array(R1)
    R2 = np.array(R2)
    center = (R1 + R2) / 2
    diff_vec = (R2 - R1)
    diff_vec_norm = np.linalg.norm(diff_vec)
    alpha = 0.66
    left_extension = R1 - alpha * diff_vec
    length = 2 * alpha + np.linalg.norm(diff_vec) 
    first_corner = left_extension + np.cross(length / 4 * diff_vec / diff_vec_norm, [0, 0, 1])
    second_corner = left_extension - (first_corner - left_extension)
    third_corner = center - (first_corner - center)
    
    width_unit_vector = first_corner - second_corner
    width_unit_vector /= width_ticks

    length_unit_vector = third_corner - second_corner
    length_unit_vector /= length_ticks
    points = [(second_corner + i * width_unit_vector + j * length_unit_vector).tolist() for i in range(width_ticks) for j in range(length_ticks)]
    return points

In [7]:
def line_coordinates(R1, R2):
    start_point = R1 - 10 * (R2 - R1)
    end_point = R2 - 10 * (R1 - R2)
    points = np.linspace(start_point, end_point, 100)
    return points

In [8]:
def cube_coordinates(R1, R2, side_length, num_ticks=10):
    coords = []
    for R in [R1, R2]:
        corner = R - side_length / 2 * np.ones((3,))
        print(R1, corner)
        x_vec = side_length * np.array([1, 0, 0]) / (num_ticks - 1)
        y_vec = np.roll(x_vec, 1)
        z_vec = np.roll(y_vec, 1)
        cube_points = [(corner + i * x_vec + j * y_vec + k * z_vec).tolist() for i in range(num_ticks) for j in range(num_ticks) for k in range(num_ticks)] 
        coords += cube_points
    coords = np.array(coords)
    return coords

In [9]:
def generate_coords(args, coord_type):
    R_data = np.load("./md_h2_R_coords.npz", allow_pickle=True)
    R_total = R_data["R"]
    coords = []
    func = eval(f"{coord_type}_coordinates")
    for R in R_total:
        [R1, R2] = R
        coords_per_R = func(*((R1, R2) + args))
        coords.append(coords_per_R)
    coords = np.array(coords)
    rectangle_data_dict = {
        "R": R_total,
        f"{coord_type}_coords": coords,
    }
    with open(f"./md_h2_R_{coord_type}_coords.npz", "wb") as f:
        pickle.dump(rectangle_data_dict, f)

In [10]:
coord_type = "rectangle"
args = (CONSTANTS["width_ticks"], CONSTANTS["length_ticks"])
generate_coords(args, coord_type)

In [19]:
def create_densities_and_corrs(length, overwrite=False, with_corrs=False, verbose=1, corr_local=True, squaring=False, local_square_size=5, coord_type='rectangle'):
    if overwrite:
        data_dict = {key: None for key in ["R", "densities", "corrs", "coords"]}
        with open("./md_h2.npz", "wb") as f:
            pickle.dump(data_dict, f)
    R_and_coords = np.load(f"./md_h2_R_{coord_type}_coords.npz", allow_pickle=True)
    R = R_and_coords["R"]
    all_coords = R_and_coords[f"{coord_type}_coords"]
    i = 0
    if length == -1:
        length = len(R)
    for i in range(length):
        [R1, R2] = R[i]
        coords = all_coords[i]
        if verbose == 1:
            print(i / length)
            i += 1
        atom_data = [[1, R1.tolist()], [1, R2.tolist()]]
        mol = gto.M()
        mol.atom = atom_data
        mol.basis = "ccpvdz"
        mol.build()
        hf = mol.RHF().run(verbose=0)
        mp2 = hf.MP2().run(verbose=0)
        rdm1 = mp2.make_rdm1(ao_repr=True)
        rdm2 = mp2.make_rdm2(ao_repr=True)
        # ao_vals: (n_coords, n_ao)
        ao_vals = np.array(mol.eval_ao("GTOval_sph", coords))
        if squaring:
            ao_vals = ao_vals.reshape(CONSTANTS["width_ticks"], CONSTANTS["length_ticks"], ao_vals.shape[-1])
            ao_vals = ao_vals.reshape(CONSTANTS["width_ticks"] // local_square_size, local_square_size, CONSTANTS["length_ticks"] // local_square_size, local_square_size, ao_vals.shape[-1])
            ao_vals = ao_vals.transpose((0, 2, 1, 3, 4))
            ao_vals = ao_vals.reshape((CONSTANTS["width_ticks"] * CONSTANTS["length_ticks"]) // (local_square_size ** 2), local_square_size, local_square_size, ao_vals.shape[-1])
            densities_for_molecule = np.einsum("ij,cxyi,cxyj->cxy", rdm1, ao_vals, ao_vals)
            if with_corrs:
                if coord_type == "cube":
                    ao_vals_new_shape = (2, ao_vals.shape[0] // 2, ao_vals.shape[1])
                    ao_vals = ao_vals.reshape(ao_vals_new_shape)
                    corrs_for_molecule = 0.5 * np.einsum("ijkl,cni,cnj,cmk,cml->cnm", rdm2, ao_vals, ao_vals, ao_vals, ao_vals)
                elif coord_type == 'rectangle' and corr_local:
                    corrs_for_molecule = 0.5 * np.einsum("ijkl,cxyi,cxyj,cabk,cabl->cxyab", rdm2, ao_vals, ao_vals, ao_vals, ao_vals)
        else:
            densities_for_molecule = np.einsum("ij,ni,nj->n", rdm1, ao_vals, ao_vals)
            if with_corrs:
                print("Not feasible in current choice of coordinates!")
        
        atom_data = [[x, y, z] for [i, [x, y, z]] in atom_data]
        temp_dict = {
            "R": np.array(atom_data, dtype=np.float32)[np.newaxis, :, :], 
            "densities": densities_for_molecule[np.newaxis, :],
            "corrs": None,
            "coords": coords[np.newaxis, :]
        }
        temp_dict["corrs"] = corrs_for_molecule[np.newaxis, :]
            

        file_name = "./md_h2.npz"
        current_data = np.load(file_name, allow_pickle=True)
        for key in ["R", "densities", "coords", "corrs"]:
            if current_data[key] is None:
                current_data[key] = temp_dict[key]
            else:
                current_data[key] = np.concatenate((current_data[key], temp_dict[key]))
        with open("./md_h2.npz", "wb") as f:
            pickle.dump(current_data, f)
        

In [22]:
create_densities_and_corrs(-1, overwrite=True, with_corrs=True)

0.0
0.0008333333333333334
0.0016666666666666668
0.0025
0.0033333333333333335
0.004166666666666667
0.005
0.005833333333333334
0.006666666666666667
0.0075
0.008333333333333333
0.009166666666666667
0.01
0.010833333333333334
0.011666666666666667
0.0125
0.013333333333333334
0.014166666666666666
0.015
0.015833333333333335
0.016666666666666666
0.0175
0.018333333333333333
0.019166666666666665
0.02
0.020833333333333332
0.021666666666666667
0.0225
0.023333333333333334
0.024166666666666666
0.025
0.025833333333333333
0.02666666666666667
0.0275
0.028333333333333332
0.029166666666666667
0.03
0.030833333333333334
0.03166666666666667
0.0325
0.03333333333333333
0.034166666666666665
0.035
0.035833333333333335
0.03666666666666667
0.0375
0.03833333333333333
0.03916666666666667
0.04
0.04083333333333333
0.041666666666666664
0.0425
0.043333333333333335
0.04416666666666667
0.045
0.04583333333333333
0.04666666666666667
0.0475
0.04833333333333333
0.049166666666666664
0.05
0.050833333333333335
0.0516666666666666

In [8]:
data = np.load("md_h2.npz", allow_pickle=True)

In [9]:
data["coords"].shape

(1200, 200, 3)

In [10]:
data["densities"].shape

(1200, 200)

In [16]:
dim_samples = data["densities"].shape[0]
width_ticks = 10
length_ticks = 20
local_square_size = 5
arr = data["densities"].reshape((dim_samples, width_ticks, length_ticks))
arr = arr.reshape((dim_samples, width_ticks // local_square_size, local_square_size, length_ticks // local_square_size, local_square_size))
arr = arr.transpose((0, 1, 3, 2, 4))
arr.shape

(1200, 2, 4, 5, 5)

In [17]:
data["corrs"].shape

(1200, 8, 5, 5, 5, 5)

In [18]:
sample_dim = data["corrs"].shape[0]
square_dim = data["corrs"].shape[1]
new_target_shape = (sample_dim * square_dim,) + tuple(data["corrs"].shape[2:])
data["corrs"].reshape(new_target_shape).shape

(9600, 5, 5, 5, 5)